# Deploy Auto-AI model image on SAP AI CORE

The following cell installs the necessary Python libraries.

In [2]:
!pip install ai-core-sdk
!pip install python-dotenv

Import the following libraries:

In [12]:
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
import os
import requests

Below cell loads environment variables from the `.env` file located within the same folder

In [4]:
# Reads .env file from the same folder
from dotenv import load_dotenv
load_dotenv()


True

### Create SAP AI Core client
Create SAP AI Core client by authenticating using the credentials stored in `.env` file

In [5]:
ai_core_client = AICoreV2Client(
# `AI_API_URL`
   base_url = os.getenv("BASEURL"),
# `URL`
   auth_url =  os.getenv("AUTHURL"),
# `clientid`
  client_id = os.getenv("CLIENTID"),
# `clientsecret`
  client_secret = os.getenv("CLIENTSECRET")
)

### List Resource Groups
List all the resource groups in the sap ai core account 

In [6]:
response = ai_core_client.resource_groups.query()
for app in response.resources:
    print(app.__dict__)

{'resource_group_id': 'vikrambhat', 'labels': [], 'status': 'PROVISIONED', 'created_at': datetime.datetime(2023, 7, 4, 16, 5, 9)}
{'resource_group_id': 'default', 'labels': [], 'status': 'PROVISIONED', 'created_at': datetime.datetime(2023, 6, 30, 9, 6, 9)}


## Add docker Registry

Create a docker registry in the sap ai core account. This is required to pull images from docker container

In [7]:
# read docker registry password stored in the .env file 
dockerregistrypassword=os.getenv("DOCKERREGISTRY")

In [8]:
response = ai_core_client.docker_registry_secrets.create(
    name = "vbrepo",
    data = {
        ".dockerconfigjson": "{\"auths\":{\"https://index.docker.io\":{\"username\":\"vikrambhat2\",\"password\":\""+dockerregistrypassword+"\"}}}"
    }
)

print(response.__dict__)


{'message': 'Secret exists'}


## Add Github Repo

In [6]:
# read github access token stored in the .env file 
github_access_token=os.getenv("GITHUBTOKEN")

In [ ]:
ai_core_client.repositories.create(
    name = "vikram-aicore-pipelines",
    url = "https://github.com/vikrambhat2/sapaicore_mlexp",
    username = "vikrambhat2",
    password = github_access_token
)

## Check github onboarding status

In [9]:
# check on-boarding status
response = ai_core_client.repositories.query()
#
for repository in response.resources:
    print('Name:', repository.name)
    print('URL:', repository.url)
    print('Status:', repository.status)

Name: vikram-aicore-pipelines
URL: https://github.com/vikrambhat2/sapaicore_mlexp
Status: RepositoryStatus.COMPLETED


### Specify the folder containing yaml file

In [10]:
yaml_path="AUTOAIEXEC"

### Read execution file(yaml) and create an application

In [9]:
## Code to delete existing application
res=ai_core_client.applications.delete(application_name="aicore-app-autoai")
res.message

'Application has been successfully deleted.'

In [10]:
github_app_name="aicore-app-autoai"
ai_core_client.applications.create(
    application_name = github_app_name,
    repository_url = "https://github.com/vikrambhat2/sapaicore_mlexp",
    path = yaml_path,
    revision = "HEAD"
)


In [11]:
response = ai_core_client.applications.query()

for app in response.resources:
    print(app.__dict__)

{'path': 'AUTOAIEXEC', 'revision': 'HEAD', 'repository_url': 'https://github.com/vikrambhat2/sapaicore_mlexp', 'application_name': 'aicore-app-autoai'}
{'path': 'DemandResponse', 'revision': 'HEAD', 'repository_url': 'https://github.com/vikrambhat2/sapaicore_mlexp', 'application_name': 'aicore-app-scikit'}
{'path': 'DemandResponseExec', 'revision': 'HEAD', 'repository_url': 'https://github.com/vikrambhat2/sapaicore_mlexp', 'application_name': 'aicore-app-vb'}


### Check if execution file sync is successful

In [12]:
response = ai_core_client.applications.get_status(application_name=github_app_name)


for workflow_sync_status in response.sync_ressources_status:
    print(workflow_sync_status.__dict__)
print(response.__dict__)
print('*'*80)
print(response.sync_ressources_status[0].__dict__)


{'name': 'auto-pipeline', 'kind': 'ServingTemplate', 'status': 'Synced', 'message': 'servingtemplate.ai.sap.com/auto-pipeline created'}
{'health_status': 'Healthy', 'sync_status': 'Synced', 'message': 'successfully synced (all tasks run)', 'source': <ai_core_sdk.models.application_source.ApplicationSource object at 0x7fea20a68d90>, 'sync_finished_at': '2023-07-26T10:41:07Z', 'sync_started_at': '2023-07-26T10:41:06Z', 'reconciled_at': '2023-07-26T10:41:07Z', 'sync_ressources_status': [<ai_core_sdk.models.application_resource_sync_status.ApplicationResourceSyncStatus object at 0x7fea20a71670>]}
********************************************************************************
{'name': 'auto-pipeline', 'kind': 'ServingTemplate', 'status': 'Synced', 'message': 'servingtemplate.ai.sap.com/auto-pipeline created'}


## Create Configuration

In [13]:
exec_id="auto-pipeline"
scen_id="demandresponse-autoai"

In [59]:
from ai_core_sdk.models import InputArtifactBinding
from ai_api_client_sdk.models.parameter_binding import ParameterBinding

response = ai_core_client.configuration.create(
    name = "demandresponse-autoai-withaicore",
    resource_group = "vikrambhat",
    scenario_id = scen_id, # value from workflow
    executable_id = exec_id,
#        input_artifact_bindings = [
#        # list
#        InputArtifactBinding(key="demandresponsemodel", artifact_id =artifact_id), # Change artifact id
#    ]

)

print(response.__dict__)

{'id': '0f2a5496-01de-409c-bf38-bceb68b8f096', 'message': 'Configuration created'}


In [60]:
conf_id=response.__dict__['id']

In [61]:
# Lists all configurations
response = ai_core_client.configuration.query(
    resource_group = "vikrambhat"
)

for configuration in response.resources:
    print(configuration.__dict__)
    


{'id': '0f2a5496-01de-409c-bf38-bceb68b8f096', 'name': 'demandresponse-autoai-withaicore', 'scenario_id': 'demandresponse-autoai', 'executable_id': 'auto-pipeline', 'parameter_bindings': [], 'input_artifact_bindings': [], 'created_at': datetime.datetime(2023, 7, 27, 16, 44, 21), 'scenario': None}
{'id': '26d1b606-fb3c-4768-b52b-876737717e1b', 'name': 'demandresponse-autoai-withaicore', 'scenario_id': 'demandresponse-autoai', 'executable_id': 'auto-pipeline', 'parameter_bindings': [], 'input_artifact_bindings': [], 'created_at': datetime.datetime(2023, 7, 27, 16, 43, 43), 'scenario': None}


## Create Deployment

In [62]:
response = ai_core_client.deployment.create(
    resource_group = 'vikrambhat',
    configuration_id = conf_id # change this
)
print(response.__dict__)
deploymentid=response.__dict__['id']

{'id': 'd4f1c9d13a71f1a6', 'message': 'Deployment scheduled.', 'deployment_url': '', 'status': <Status.UNKNOWN: 'UNKNOWN'>, 'ttl': None}


## Check deployment status

In [72]:
response = ai_core_client.deployment.get(
    resource_group = 'vikrambhat',
    deployment_id = deploymentid # Change this
)

print("Status: ", response.status)
print('*'*80)
print(response.__dict__)


Status:  Status.RUNNING
********************************************************************************
{'id': 'd4f1c9d13a71f1a6', 'configuration_id': '0f2a5496-01de-409c-bf38-bceb68b8f096', 'configuration_name': 'demandresponse-autoai-withaicore', 'scenario_id': 'demandresponse-autoai', 'status': <Status.RUNNING: 'RUNNING'>, 'target_status': <TargetStatus.RUNNING: 'RUNNING'>, 'created_at': datetime.datetime(2023, 7, 27, 16, 44, 27), 'modified_at': datetime.datetime(2023, 7, 27, 16, 46, 50), 'status_message': None, 'status_details': None, 'submission_time': datetime.datetime(2023, 7, 27, 16, 44, 57), 'start_time': datetime.datetime(2023, 7, 27, 16, 46, 43), 'completion_time': None, 'deployment_url': 'https://api.ai.prod.us-east-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d4f1c9d13a71f1a6', 'last_operation': <Operation.CREATE: 'CREATE'>, 'latest_running_configuration_id': '0f2a5496-01de-409c-bf38-bceb68b8f096', 'ttl': None}


In [68]:
from ai_core_sdk.models import TargetStatus
response=ai_core_client.deployment.query_logs(resource_group = 'vikrambhat',
    deployment_id = deploymentid)
for log in response.data.result:
    print(log.msg)
    print("---")

Starting
---
[2023-07-27 16:45:22 +0000] [7] [INFO] Starting gunicorn 21.2.0
---
[2023-07-27 16:45:22 +0000] [7] [INFO] Listening at: http://0.0.0.0:7000 (7)
---
[2023-07-27 16:45:22 +0000] [7] [INFO] Using worker: sync
---
[2023-07-27 16:45:22 +0000] [8] [INFO] Booting worker with pid: 8
---


## Make prediction

In [11]:
deployment_url = "https://api.ai.prod.us-east-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d4f1c9d13a71f1a6"


In [13]:


# URL


endpoint = f"{deployment_url}/v2/greet" # endpoint implemented in serving engine
headers = {"Authorization": ai_core_client.rest_client.get_token(),
           'ai-resource-group': "vikrambhat"}
response = requests.get(endpoint, headers=headers)

print(response.text)

Demand Response Model is loaded.


In [14]:

# Preparing the input for inference
test_input={'input_data': [{'fields': ['CUSTOMER_ID','CSTFNM','CSTLNM','CSTPH1','CSTML','AGE','CITY','MARITAL_STATUS','GENDER','EDUCATION','EMPLOYMENT','TENURE','SEGMENT','HOME_SIZE','ENERGY_USAGE_PER_MONTH','ENERGY_EFFICIENCY','IS_REGISTERED_FOR_ALERTS','OWNS_HOME','COMPLAINTS','EST_INCOME','CLTV','HAS_THERMOSTAT','HAS_HOME_AUTOMATION','PHOTOVOLTAIC_ZONING','WIND_ZONING','SMART_METER_COMMENTS','IS_CAR_OWNER','HAS_EV','HAS_PHOTOVOLTAIC','HAS_WIND','EBILL','IN_WARRANTY','STD_YRLY_USAGE','MISSED_PAYMENT','YEARLY_USAGE_PREDICTED'],
   'values': [[1,'Leonor','Cummerata','781-757-3537','Leonor.Cummerata@nola.biz',44,'Mountain View','S','male','Associate degree','Employed full-time',13,'Green Advocate',1980,5080,0.39,True,True,False,47870,352.0,True,False,True,False,'Negative',True,True,True,False,False,True,56049,False,59972.4]]}]}

endpoint = f"{deployment_url}/v2/predict" # endpoint implemented in serving engine
headers = {"Authorization": ai_core_client.rest_client.get_token(),
           'ai-resource-group': "vikrambhat",
           "Content-Type": "application/json"}
response = requests.post(endpoint, headers=headers, json=test_input)

print( response.json())

{'predictions': [{'fields': ['prediction', 'probability'], 'values': [[0.0, [0.6210089111178061, 0.37899108888219385]]]}]}
